# 🎯 상품 속성 추출을 위한 프롬프트 엔지니어링

## 📋 워크샵 개요

이번 실습에서는 **의류 상품 이미지**를 활용하여 속성을 추출하기 위한 프롬프트를 단계적으로 개선해나가는 과정을 학습합니다.

생성형 AI의 강력한 이미지 분석 능력을 실제 비즈니스 환경에서 어떻게 활용할 수 있는지 직접 체험해보겠습니다.

## 🎯 학습 목표

### 1. 📊 기본적인 프롬프트의 한계점 이해
- 제약 없는 프롬프트의 문제점 파악
- 일관성 부족으로 인한 실무 적용의 어려움

### 2. 🎛️ 구체적인 속성 지정을 통한 결과 개선
- 원하는 정보만 추출하는 방법
- JSON 형식을 통한 구조화된 데이터 확보

### 3. 🔒 제약 조건을 통한 일관성 확보
- 키워드 제한을 통한 표준화
- 데이터 품질 관리 방법

### 4. ⚖️ 구조화된 JSON 출력으로 데이터 정규화
- 하이브리드 접근법을 통한 최적 솔루션
- 실무 활용 가능한 실용적 구현

## ⏱️ 예상 소요 시간: 30분

---

*"효과적인 프롬프트 엔지니어링은 명확한 목표와 체계적인 접근에서 시작됩니다."*

In [ ]:
# 필요 라이브러리 설치
%pip install -r requirements.txt

In [33]:
# 초기 설정
import boto3
import json
from IPython.display import Image, display
from pathlib import Path

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)

CLAUDE_3_5_SONNET = 'us.anthropic.claude-3-5-sonnet-20240620-v1:0'
CLAUDE_3_5_HAIKU = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
image_directory = Path("images")
image_paths = [str(file.resolve()) for file in image_directory.iterdir()]

In [ ]:
# 테스트할 이미지 로드 및 확인
def load_image(image_path):
    with open(image_path, "rb") as image_file:
        return image_file.read()

# Bedrock 호출 테스트
def call_bedrock(messages, model=CLAUDE_3_5_HAIKU):
    try:
        response = bedrock_runtime.converse(
            modelId=model,
            messages=messages
        )

        return response.get("output").get("message").get("content")[0].get("text")
    
    except Exception as e:
        print(f"Error calling Bedrock: {e}")
        return None
    
prompt = "Hello, Claude!"

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": prompt
            }
        ]
    }
]

response = call_bedrock(messages)

response

# 1️⃣ 단계: 제약 없는 기본 프롬프트

## 🚀 시작점: 가장 단순한 접근법

가장 기본적인 형태의 프롬프트부터 시작해보겠습니다. 이 단계에서는 구체적인 지시사항 없이 단순하게 **속성을 추출해달라**고 요청합니다.

### 🤔 예상되는 문제점

| 문제점 | 설명 | 비즈니스 영향 |
|--------|------|---------------|
| **일관성 부족** | 같은 상품도 실행할 때마다 다른 결과 | 데이터 신뢰성 저하 |
| **불필요한 정보** | 원하지 않는 속성까지 추출 | 후처리 비용 증가 |
| **누락 가능성** | 중요한 속성이 빠질 수 있음 | 정보 손실 |

### 💡 학습 포인트
이 단계를 통해 **생성형 AI의 기본 능력**을 확인하고, 동시에 **구조화되지 않은 프롬프트의 한계**를 직접 경험해보겠습니다.

---

In [ ]:
# 1단계: 기본 프롬프트

prompt = "다음 이미지를 보고 속성을 추출해줘"

def basic_prompt(image_path, prompt):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": "png",
                        "source": {
                            "bytes": load_image(image_path)
                        }
                    }
                },
                {
                    "text": prompt
                }
            ]
        }
    ]
    display(Image(image_path, width=200, height=200))
    response = call_bedrock(messages, model=CLAUDE_3_5_SONNET)
    return response

response = basic_prompt("images/01_basic_white_tshirt.png", prompt)

print(response)

## 📊 JSON 형식의 중요성

일반적으로 **데이터를 처리**하기 위해서는 구조화된 형식이 필요합니다. JSON 형식을 사용하면 다음과 같은 장점이 있습니다:

### ✅ JSON 출력의 장점

- **🤖 기계 판독 가능**: 프로그래밍적 처리 용이
- **💾 데이터베이스 저장**: 직접적인 DB 삽입 가능  
- **🔗 API 활용**: 웹 서비스 응답으로 바로 사용
- **⚙️ 자동화 처리**: 후처리 워크플로우 구축 가능

### 🎯 실습 목표
기본 프롬프트에 **JSON 출력 제약 조건**을 추가하여 구조화된 데이터 추출을 시도해보겠습니다.

In [ ]:
prompt = "다음 이미지를 보고 속성을 추출해줘. 출력은 JSON 형식으로 해줘. JSON 외에는 다른 어떠한 텍스트도 포함하지 말아줘."

response = basic_prompt("images/01_basic_white_tshirt.png", prompt)

# 출력 결과를 JSON 형식으로 변환
json_results = json.loads(response)

# 출력 변환 및 결과 확인
print(f"타입: {type(json_results)}")
json_results

JSON 출력 형식을 활용하면 데이터를 구조화된 형태로 추출하여 코드에 활용하거나 데이터베이스에 저장하기에 용이해집니다.

In [ ]:
print("기본 흰색 무지티 속성 추출 결과")

for feature, value in json_results.items():
    print(f"속성: {feature}, 값: {value}")

이제 여러 이미지에 동일한 프롬프트를 적용해서 **일관성**을 확인해보겠습니다.

### 🎯 확인 포인트
- 각 이미지별 결과 형식의 통일성
- 속성 추출의 정확도
- 예상치 못한 변수들의 발견

In [ ]:
for image_path in sorted(image_paths):
    print(basic_prompt(image_path, prompt))

## 📈 1단계 결과 분석

### 🔍 관찰된 문제점

모든 상품 이미지들에 대한 결과를 확인해보셨나요? 

추출한 속성은 **정확하지만 형식이 제각각**인 것을 확인할 수 있습니다:

| 발견된 문제 | 예시 |
|-------------|------|
| **용어 불일치** | "흰색" vs "화이트" |
| **형식 차이** | 단일값 vs 배열 | 
| **세부도 차이** | "줄무늬" vs "가로 줄무늬" |

### 💡 핵심 인사이트

> **생성형 AI 모델은 확률 기반으로 답변을 구성합니다.**
> 
> 따라서 원하는 답변을 얻기 위해서는 적절한 **제약 조건**을 제공하는 것이 중요합니다.

### 🎯 다음 단계 예고
위에서 사용한 `"출력은 JSON 형식으로 해줘"` 같은 지시사항이 바로 **제약 조건**의 한 예입니다.

다음 단계에서는 **형식의 일관성을 보장**하는 고급 프롬프팅 기법을 학습하겠습니다.

# 2️⃣ 단계: 내가 원하는 속성 추출하기

## 🎯 목표: 양식 통일성 확보

1단계에서는 생성형 AI의 **속성 추출 정확도**는 확인했지만, **양식의 통일성 부족**이라는 과제를 발견했습니다.

### ❓ 해결 과제
> 양식의 통일성을 확보하기 위한 프롬프트 작성 방법은?

## 🔧 해결 방법: 제약 조건 설정

구체적인 속성들을 **명시적으로 지정**하여 더 일관된 결과를 얻어보겠습니다.

### 📋 추출할 속성 목록

| 속성 | 영문 | 예시 값 |
|------|------|---------|
| **종류** | Type | 티셔츠, 셔츠, 블라우스 |
| **색상** | Color | 검정, 흰색, 빨강 |
| **패턴** | Pattern | 무지, 스트라이프, 체크 |
| **핏** | Fit | 슬림핏, 레귤러핏, 오버사이즈 |
| **스타일** | Style | 캐주얼, 포멀, 스포츠 |
| **넥라인** | Neckline | 라운드넥, V넥, 터틀넥 |
| **소매 길이** | Sleeve Length | 민소매, 반소매, 긴소매 |

### 💡 기대 효과
- ✅ **누락 방지**: 원하는 모든 속성 추출 보장
- ✅ **형식 통일**: 일관된 JSON 구조
- ✅ **처리 효율**: 예측 가능한 데이터 형태

In [ ]:
# 2단계: 내가 원하는 속성 추출하기

prompt = """다음 이미지를 보고 속성을 추출해줘. 출력은 JSON 형식으로 해줘. JSON 외에는 다른 어떠한 텍스트도 포함하지 말아줘.
다음 속성들을 추출해줘.
- 종류 (Type)
- 색상 (Color)
- 패턴 (Pattern)
- 핏 (Fit)
- 스타일 (Style)
- 넥라인 (Neckline)
- 소매 길이 (Sleeve length)
답은 한글로 작성해줘"""

for image_path in sorted(image_paths)[:3]:
    print(basic_prompt(image_path, prompt))

1단계에서와 달리 우리가 원하는 속성 여덟 종류에 한해서 답을 잘 구성해주는 것을 확인할 수 있습니다.

여러분의 결과물과 다를 수 있지만 현재 테스트 결과는 다음과 같이 나왔습니다.

**흰색 반팔 티셔츠**
```json
{
  "종류": "티셔츠",
  "색상": "흰색",
  "패턴": "무지",
  "핏": "일반핏",
  "스타일": "베이직",
  "넥라인": "라운드넥",
  "소매 길이": "반팔"
}
```

**줄무늬 긴팔 티셔츠**
```json
{
  "종류": "티셔츠",
  "색상": "네이비와 화이트",
  "패턴": "가로 줄무늬",
  "핏": "레귤러 핏",
  "스타일": "캐주얼",
  "넥라인": "라운드넥",
  "소매 길이": "긴소매"
}
```

**빨간색 체크무늬 티셔츠**
```json
{
  "종류": "셔츠",
  "색상": "빨간색과 검은색",
  "패턴": "체크무늬",
  "핏": "일반핏",
  "스타일": "캐주얼",
  "넥라인": "버튼다운 칼라",
  "소매 길이": "긴소매"
}
```

결과물을 확인해보면 여러 색깔이 있을 경우 우리가 데이터 처리를 용이하게 하기 위해서는 "네이비와 화이트", "빨간색과 검은색" 같은 단어를 사용하는 대신 List 데이터 타입을 활용하면 좋을 것 같습니다.

이를 위해 프롬프트를 다시 조정해보겠습니다.

In [ ]:
prompt = """다음 이미지를 보고 속성을 추출해줘. 출력은 JSON 형식으로 해줘. JSON 외에는 다른 어떠한 텍스트도 포함하지 말아줘.

다음 속성들을 추출하고 답은 한글로 해줘.
- 종류 (Type)
- 색상 (Color)
- 패턴 (Pattern)
- 핏 (Fit)
- 스타일 (Style)
- 넥라인 (Neckline)
- 소매 길이 (Sleeve length)
- 소재 (Material)

만약 색상이 여러 가지 있을 경우 색상 항목에 리스트 형태로 답을 작성해줘"""

for image_path in sorted(image_paths)[:3]:
    print(basic_prompt(image_path, prompt))

## 📊 2단계 결과 분석

### ✅ 개선된 점들

1단계와 달리 **지정한 속성에 한해서만** 답변이 구성되는 것을 확인할 수 있습니다!

### 🎯 예시 결과 비교

**🔵 흰색 반팔 티셔츠**
```json
{
  "종류": "티셔츠",
  "색상": "흰색", 
  "패턴": "무지",
  "핏": "일반핏",
  "스타일": "베이직",
  "넥라인": "라운드넥",
  "소매 길이": "반팔"
}
```

**🔵 줄무늬 긴팔 티셔츠**
```json
{
  "종류": "티셔츠",
  "색상": ["네이비", "화이트"],
  "패턴": "가로 줄무늬", 
  "핏": "레귤러 핏",
  "스타일": "캐주얼",
  "넥라인": "라운드넥",
  "소매 길이": "긴소매"
}
```

### 🔍 발견된 새로운 과제
#### 여전한 동의어 문제 ⚠️
| 속성 | 동의어 예시 | 문제점 |
|------|-------------|--------|
| 색상 | "흰색" vs "화이트" | 검색/필터링 혼란 |
| 핏 | "일반핏" vs "레귤러 핏" | 카테고리 중복 |
| 스타일 | "베이직" vs "캐주얼" | 분류 기준 모호 |


### 🎯 다음 단계
동의어 발생 방지를 위한 제약 조건 설정 방법을 학습하겠습니다.

# 3️⃣ 단계: 키워드 제한하기

## 🎯 목표: 동의어 문제 해결

사용자 입장에서 **동의어가 늘어나는 것**은 시스템 관리에 큰 부담이 됩니다.

### ⚠️ 동의어로 인한 문제점
- **검색 효율성 저하**: "흰색"과 "화이트" 별도 처리 필요
- **데이터 정합성 문제**: 같은 의미, 다른 표현으로 인한 혼란
- **유지보수 비용 증가**: 지속적인 용어 통합 작업 필요

## 🔧 해결책: 키워드 제한 프롬프팅

**미리 정의된 키워드 목록**을 제공하여 AI가 해당 범위 내에서만 답변하도록 제약합니다.

## ✅ 키워드 제한의 장점

### 1. **일관성 및 표준화** 🎯
- 모든 상품이 동일한 용어 체계 사용
- 데이터 분석과 검색 작업 간소화  
- 애매한 표현으로 인한 혼란 방지

### 2. **품질 관리** 🛡️
- 부적절하거나 창의적인 응답 방지
- AI 응답 검증 프로세스 단순화
- 후처리 작업 부담 대폭 감소

## 🎯 실습 목표
정의된 키워드 목록을 활용하여 **완전히 표준화된** 속성 추출을 구현해보겠습니다.

In [ ]:
keywords = {
    "종류": ["티셔츠", "셔츠", "블라우스", "후드티", "스웨터", "니트", "탱크톱", "크롭탑", "튜닉", "카디건", "기타"],
    "색상": ["검정", "흰색", "회색", "빨강", "파랑", "노랑", "초록", "보라", "분홍", "갈색", "베이지", "네이비", "주황", "기타"],
    "패턴": ["무지", "스트라이프", "체크", "기타"],
    "핏": ["슬림핏", "레귤러핏", "루즈핏", "오버사이즈", "기타"],
    "스타일": ["캐주얼", "포멀", "스포츠", "빈티지", "기타"],
    "넥라인": ["라운드넥", "브이넥", "터틀넥", "기타"],
    "소매 길이": ["민소매", "반소매", "긴소매", "기타"],
}

prompt = f"""다음 이미지를 보고 속성을 추출해줘. 출력은 JSON 형식으로 해줘. JSON 외에는 다른 어떠한 텍스트도 포함하지 말아줘.

다음 속성들을 추출하되 목록에 있는 키워드만 활용하고 답은 한글로 해줘.
{keywords}

만약 색상이 여러 가지 있을 경우 색상 항목에 리스트 형태로 답을 작성해줘"""

for image_path in sorted(image_paths)[:3]:
    print(basic_prompt(image_path, prompt))

키워드들이 잘 적용되었는지 확인해보겠습니다.

추가로, 아래 과정은 속성 추출 작업 이후 후처리 여부를 판단하는 데에도 사용할 수 있습니다.

In [ ]:
#  속성 추출 결과에 사용하지 않는 키워드가 발견되었는지 검사
def evaluate_reponse(response, keywords):
    incorrect_count = 0
    json_results = json.loads(response)
    for feature, value in json_results.items():
        if feature in keywords:
            if isinstance(value, list):
                if not all(item in keywords[feature] for item in value):
                    incorrect_count += 1
            else:
                if value not in keywords[feature]:
                    incorrect_count += 1
        else:
            incorrect_count += 1
    return incorrect_count

for image_path in sorted(image_paths):
    response = basic_prompt(image_path, prompt)
    print(response)
    incorrect_count = evaluate_reponse(response, keywords)
    print(f"{incorrect_count}개의 사전에 없는 키워드가 발견되었습니다.")

## 📊 3단계 결과 분석 및 검증

### ✅ 성과 확인

키워드 제한이 성공적으로 적용되었는지 **자동 검증 시스템**을 통해 확인해보았습니다.

### 🤔 새로운 과제 발견

키워드 제한이 성공적으로 작동했지만, **새로운 문제**도 발견됩니다:

#### ⚠️ 의미 손실 문제
- **패턴 속성**: 대부분이 "기타"로 분류
- **색상 뉘앙스**: "버건디", "와인색" → 단순히 "빨강"
- **표현력 제약**: 풍부한 설명이 획일적으로 변화

### 💭 근본적 질문
> 키워드를 제한하는 것이 항상 최선의 해결책일까요?

#### 🔴 키워드 제한의 단점

1. **뉘앙스의 손실**
   - 미묘한 색상 차이 무시
   - 브랜드 고유의 표현 방식 상실

2. **예외 상황 처리 어려움**  
   - 새로운 트렌드나 디자인에 대응 불가
   - 혁신적인 제품의 특징 표현 제약

3. **사용자 경험 제약**
   - 고객의 다양한 검색어와 시스템 용어 불일치
   - 상세한 제품 설명의 획일화

### 🎯 다음 단계
1, 2단계와 3단계의 **장단점을 결합**하는 하이브리드 방식의 접근법

# 4️⃣ 단계: 키워드와 의미를 복합적으로 추출하는 프롬프팅

## 하이브리드 접근법

이전 단계에서 **제한된 키워드**를 통한 표준화의 장점과 **표현력 상실**이라는 단점을 모두 확인했습니다.

### 💡 핵심 아이디어: 두 마리 토끼를 모두 잡기

현재 10개의 상품만 다루고 있지만, 실제로는 **수천, 수만 개의 다양한 상품**을 처리해야 합니다. 이때 필요한 것은:

- ✅ **시스템 효율성** (표준 카테고리)
- ✅ **표현의 풍부함** (상세 설명)

## 🔧 하이브리드 접근법의 구조

### 📊 이중 구조 정보 추출

```json
{
  "색상": {
    "카테고리": "빨강",
    "상세": "깊은 버건디 톤의 와인 레드"
  }
}
```

## 하이브리드 접근법의 장점
1. 🎯 시스템 효율성 확보
- 표준 카테고리로 일관된 데이터 구조 유지

2. 🎨 표현의 풍부함 보존
- 고객에게 매력적이고 상세한 정보 제공

3. 📈 확장성과 유연성
- 카테고리 점진적 확장 가능

In [ ]:
prompt = f"""다음 이미지를 보고 속성을 추출해줘. 출력은 JSON 형식으로 해줘. JSON 외에는 다른 어떠한 텍스트도 포함하지 말아줘.

목록에 있는 키워드만 활용해서 각 필드에 맞는 카테고리를 추출하고 상세 설명을 작성해줘.
{keywords}

만약 색상이 여러 가지 있을 경우 색상 항목에 리스트 형태로 답을 작성해줘

답변 예시:
{{
  "종류": {{
    "카테고리": "티셔츠",
    "상세": ""
  }},
  "색상": {{
    "카테고리": "빨강",
    "상세": "깊은 버건디 톤의 와인 레드"
  }},
  "패턴": {{
    "카테고리": "스트라이프", 
    "상세": "세로 방향의 얇은 핀스트라이프, 약 2mm 간격"
  }},
  "핏": {{
    "카테고리": "슬림핏",
    "상세": "몸에 밀착되는 슬림한 핏"
  }},
  "스타일": {{
    "카테고리": "캐주얼",
    "상세": "데일리 캐주얼부터 세미 포멀까지 연출 가능한 베이직 스타일"
  }},
  "넥라인": {{
    "카테고리": "라운드넥",
    "상세": "목둘레에 맞춰 둥글게 넥을 만든 넥라인"
  }},
  "소매 길이": {{
    "카테고리": "반소매",
    "상세": "소매가 반팔 길이로 줄어든 소매"
  }}
}}"""

print(basic_prompt(sorted(image_paths)[-1], prompt))

결과물을 보면 이전에 "기타"로 작성되었던 "패턴" 항목의 카테고리는 그대로 "기타"이지만 "상세" 내용을 보면 의미가 소실되지 않고 원하는 결과를 얻은 것을 볼 수 있습니다.

이제 전체 결과물에 대한 점검을 해보겠습니다.

In [ ]:
#  속성 추출 결과에 사용하지 않는 키워드가 발견되었는지 검사
def evaluate_reponse(response, keywords):
    incorrect_count = 0
    json_results = json.loads(response)
    for feature, value in json_results.items():
        if feature in keywords:
            if isinstance(value["카테고리"], list):
                if not all(item in keywords[feature] for item in value["카테고리"]):
                    incorrect_count += 1
            else:
                if value["카테고리"] not in keywords[feature]:
                    incorrect_count += 1
    return incorrect_count

for image_path in sorted(image_paths):
    response = basic_prompt(image_path, prompt)
    print(response)
    incorrect_count = evaluate_reponse(response, keywords)
    print(f"{incorrect_count}개의 사전에 없는 키워드가 발견되었습니다.")

## 🎊 4단계 최종 결과 분석

### ✅ 하이브리드 방식의 성과

결과를 확인해보면 이전에 **"기타"로만 분류**되었던 항목들도 이제 의미를 잃지 않고 정보를 보존하는 것을 확인할 수 있습니다!

#### 🔍 "패턴" 속성 개선 사례
- **카테고리**: "기타" (시스템 분류, 키워드 검색)
- **상세**: "무지개색 나선형 타이다이 패턴" (의미 기반 검색, 사용자 경험)

### 🔧 최종 품질 검증

하이브리드 방식에서도 **카테고리 키워드 준수**를 확인할 수 있습니다.

### 🚀 실제 도입 시 고려사항

#### ⚙️ 시스템 통합
- **데이터베이스 스키마**: 카테고리와 상세 필드 설계

#### 📈 지속적 개선
- **새 카테고리 발견**: 상세 설명 분석을 통한 트렌드 포착
- **품질 모니터링**: "기타" 비율 추적 및 개선
- **사용자 피드백**: 검색 패턴 분석을 통한 카테고리 확장

---

# 워크샵 결론

## 📝 학습 내용 요약

이번 워크샵에서는 의류 상품 이미지에서 속성을 추출하기 위한 프롬프트 엔지니어링 기법을 단계적으로 학습했습니다.

### 🔄 4단계 프롬프트 개선 과정

**1단계: 제약 없는 기본 프롬프트**
- ✅ 생성형 AI의 기본적인 속성 추출 능력 확인
- ❌ 결과의 일관성 부족, 양식의 통일성 문제

**2단계: 내가 원하는 속성 추출하기**
- ✅ 구체적인 속성 지정으로 원하는 정보만 추출
- ✅ JSON 형식 출력으로 구조화된 데이터 확보
- ❌ 동의어 발생 문제 (흰색/화이트, 일반핏/레귤러핏)

**3단계: 키워드 제한하기**
- ✅ 일관성 및 표준화 달성
- ✅ 데이터베이스 통합과 품질 관리 용이
- ❌ 뉘앙스 손실과 표현의 획일화

**4단계: 하이브리드 접근법**
- ✅ 카테고리(표준화) + 상세설명(풍부한 표현) 결합
- ✅ 시스템 효율성과 표현력 동시 확보
- ✅ 실제 비즈니스 환경에서 활용 가능한 실용적 해결책

## 🎯 핵심 학습 포인트

### 1. 제약 조건의 중요성
생성형 AI는 확률 기반으로 답변을 생성하므로, **명확한 제약 조건**을 통해 일관된 결과를 얻는 것이 중요합니다.

### 2. 균형잡힌 접근법
완전한 자유도와 완전한 제약 사이의 **적절한 균형점**을 찾는 것이 실무에서 가장 효과적입니다.

### 3. 비즈니스 요구사항 고려
프롬프트 설계 시 **기술적 구현 가능성**과 **비즈니스 활용도**를 모두 고려해야 합니다.

## 🚀 실무 적용 가이드

### 권장 구현 단계
1. **기본 프롬프트**로 AI 성능 확인
2. **구체적 속성 지정**으로 원하는 정보 추출
3. **키워드 제한**으로 일관성 확보
4. **하이브리드 방식**으로 최종 구현

### 활용 시나리오
- **E-commerce**: 상품 자동 카테고리화 및 상세 설명 생성
- **검색 최적화**: 카테고리 기반 필터링 + 의미론적 검색

## 💡 추가 고려사항

### 성능 최적화
- **모델 선택**: 비용 vs 성능 고려 (Haiku vs Sonnet)
- **배치 처리**: 대량 이미지 처리 시 효율성 확보
- **캐싱 전략**: 중복 처리 방지

### 품질 관리
- **검증 로직**: 키워드 준수 여부 자동 확인
- **예외 처리**: "기타" 카테고리 비율 모니터링
- **지속적 개선**: 새로운 패턴 발견 시 카테고리 확장

### 확장 가능성
- **다국어 지원**: 지역별 용어 체계 구축
- **도메인 확장**: 의류 외 다른 상품군 적용
- **개인화**: 브랜드별 특수 용어 반영

## 🎉 워크샵 완료!

축하합니다! 이제 여러분은 생성형 AI를 활용한 상품 속성 추출 시스템을 구축할 수 있는 실무 역량을 갖추게 되었습니다.

**다음 단계**: 오늘 학습한 내용을 바탕으로 실제 프로젝트에 적용해보시고, 도메인별 특성에 맞게 프롬프트를 커스터마이징해보세요!

---

*"좋은 프롬프트는 명확한 지시사항과 적절한 제약 조건의 조화에서 나옵니다."*